# New

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules, apriori

# Reading the data

In [5]:
sales_reciepts = pd.read_csv(r'C:\Users\vaish\Desktop\chatbot_LLM_app\python_code\dataset\201904 sales reciepts.csv')

In [6]:
sales_reciepts.head()

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.50,2.50,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.00,3.50,N
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,2,5.00,2.50,N
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,2,5.00,2.50,N
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,1,2.45,2.45,N


In [7]:
product = pd.read_csv(r'C:\Users\vaish\Desktop\chatbot_LLM_app\python_code\dataset\product.csv')

In [8]:
product.head()

,product_id,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,1,Whole Bean/Teas,Coffee beans,Organic Beans,Brazilian - Organic,It's like Carnival in a cup. Clean and smooth.,12 oz,14.40,$18.00,Y,N,N
1,2,Whole Bean/Teas,Coffee beans,House blend Beans,Our Old Time Diner Blend,Out packed blend of beans that is reminiscent ...,12 oz,14.40,$18.00,Y,N,N
2,3,Whole Bean/Teas,Coffee beans,Espresso Beans,Espresso Roast,Our house blend for a good espresso shot.,1 lb,11.80,$14.75,Y,N,N
3,4,Whole Bean/Teas,Coffee beans,Espresso Beans,Primo Espresso Roast,Our primium single source of hand roasted beans.,1 lb,16.36,$20.45,Y,N,N
4,5,Whole Bean/Teas,Coffee beans,Gourmet Beans,Columbian Medium Roast,A smooth cup of coffee any time of day.,1 lb,12.00,$15.00,Y,N,N


# Data wrangling

## Merge data

In [10]:
sales_reciepts = sales_reciepts[["transaction_id","transaction_date","sales_outlet_id","customer_id","product_id", "quantity"]]
product = product[["product_id","product_category","product"]]

dataset = pd.merge(sales_reciepts, product, on="product_id", how="left")
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
0,7,2019-04-01,3,558,52,1,Tea,Traditional Blend Chai Rg
1,11,2019-04-01,3,781,27,2,Coffee,Brazilian Lg
2,19,2019-04-01,3,788,46,2,Tea,Serenity Green Tea Rg
3,32,2019-04-01,3,683,23,2,Coffee,Our Old Time Diner Blend Rg
4,33,2019-04-01,3,99,34,1,Coffee,Jamaican Coffee River Sm


## Remove sizes (Rg, Lg, Sm)

In [11]:
dataset[dataset["product"].str.contains("Dark chocolate")]["product"].unique()

array(['Dark chocolate Lg', 'Dark chocolate Rg', 'Dark chocolate'],
      dtype=object)

In [14]:
dataset["product"].nunique()

80

In [16]:
dataset["product"] = dataset["product"].str.replace(' Rg', '')
dataset["product"] = dataset["product"].str.replace(' Sm', '')
dataset["product"] = dataset["product"].str.replace(' Lg', '')

In [17]:
dataset["product"].nunique()

45

In [18]:
dataset["product"].unique()

array(['Traditional Blend Chai', 'Brazilian', 'Serenity Green Tea',
       'Our Old Time Diner Blend', 'Jamaican Coffee River', 'Ethiopia',
       'English Breakfast', 'Sustainably Grown Organic', 'Earl Grey',
       'Cappuccino', 'Espresso shot', 'Latte', 'Dark chocolate',
       'Columbian Medium Roast', 'Oatmeal Scone', 'Morning Sunrise Chai',
       'Peppermint', 'Jumbo Savory Scone', 'Lemon Grass',
       'Chocolate Chip Biscotti', 'Spicy Eye Opener Chai',
       'Ginger Biscotti', 'Chocolate Croissant', 'Hazelnut Biscotti',
       'Cranberry Scone', 'Scottish Cream Scone ', 'Croissant',
       'Almond Croissant', 'Ginger Scone', 'Ouro Brasileiro shot',
       'Organic Decaf Blend', 'Chocolate syrup', 'Hazelnut syrup',
       'Carmel syrup', 'Sugar Free Vanilla syrup',
       'Jamacian Coffee River', 'Guatemalan Sustainably Grown',
       'Civet Cat', 'Chili Mayan', 'Primo Espresso Roast',
       'Brazilian - Organic', 'I Need My Bean! Diner mug',
       'Espresso Roast', 'I Need 

## Choose product subset

In [19]:
products_to_take = ['Cappuccino', 'Latte', 'Espresso shot',  \
                     'Dark chocolate','Sugar Free Vanilla syrup', 'Chocolate syrup',\
                    'Carmel syrup', 'Hazelnut syrup', 'Ginger Scone',  \
                    'Chocolate Croissant', 'Jumbo Savory Scone', 'Cranberry Scone', 'Hazelnut Biscotti',\
                    'Croissant', 'Almond Croissant', 'Oatmeal Scone', 'Chocolate Chip Biscotti',\
                    'Ginger Biscotti',\
                   ]

In [20]:
dataset = dataset[dataset["product"].isin(products_to_take)]

In [21]:
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot
20,127,2019-04-01,3,116,41,2,Coffee,Cappuccino
21,134,2019-04-01,3,189,38,2,Coffee,Latte
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino


In [22]:
dataset[["product", "product_category"]].drop_duplicates().reset_index(drop=True)

,product,product_category
0,Cappuccino,Coffee
1,Espresso shot,Coffee
2,Latte,Coffee
3,Dark chocolate,Drinking Chocolate
4,Oatmeal Scone,Bakery
5,Jumbo Savory Scone,Bakery
6,Chocolate Chip Biscotti,Bakery
7,Ginger Biscotti,Bakery
8,Chocolate Croissant,Bakery
9,Hazelnut Biscotti,Bakery


## Clean Transactions

In [ ]:
dataset.head()
# Same transaction ids can go to multiple cistomers.

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot
20,127,2019-04-01,3,116,41,2,Coffee,Cappuccino
21,134,2019-04-01,3,189,38,2,Coffee,Latte
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino


In [24]:
dataset["transaction"] = dataset["transaction_id"].astype(str) + "_" + dataset["customer_id"].astype(str)

In [26]:
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product,transaction
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino,108_65
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot,112_90
20,127,2019-04-01,3,116,41,2,Coffee,Cappuccino,127_116
21,134,2019-04-01,3,189,38,2,Coffee,Latte,134_189
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino,135_131


In [27]:
num_of_items_for_each_transaction = dataset["transaction"].value_counts().reset_index()
num_of_items_for_each_transaction.head()

,transaction,count
0,209_0,31
1,206_0,30
2,204_0,27
3,208_0,25
4,203_0,24


In [28]:
num_of_items_for_each_transaction[num_of_items_for_each_transaction["count"]==1]

,transaction,count
2646,471_8410,1
2647,1011_0,1
2648,341_5545,1
2649,1627_8289,1
2650,328_5635,1
...,...,...
8381,135_523,1
8382,130_157,1
8383,121_465,1
8384,118_748,1


In [29]:
valid_transactions = num_of_items_for_each_transaction[num_of_items_for_each_transaction["count"]>1]["transaction"].tolist()
valid_transactions[:10]

['209_0',
 '206_0',
 '204_0',
 '208_0',
 '203_0',
 '207_0',
 '205_0',
 '151_0',
 '323_0',
 '301_0']

In [31]:
dataset = dataset[dataset["transaction"].isin(valid_transactions)]

In [32]:
dataset.shape

(10189, 9)

# Product Trends

In [33]:
dataset["product"].value_counts()

product
Cappuccino                  1290
Latte                       1256
Dark chocolate               969
Chocolate Croissant          636
Espresso shot                628
Sugar Free Vanilla syrup     605
Chocolate syrup              568
Carmel syrup                 561
Hazelnut syrup               512
Ginger Scone                 417
Jumbo Savory Scone           357
Croissant                    355
Chocolate Chip Biscotti      352
Cranberry Scone              350
Almond Croissant             347
Hazelnut Biscotti            338
Oatmeal Scone                334
Ginger Biscotti              314
Name: count, dtype: int64

## Popularity Recommendation Engine

In [37]:
product_recommendation = dataset.groupby(["product", "product_category"]).count().reset_index()

In [38]:
product_recommendation = product_recommendation[["product", "product_category", "transaction_id"]]
product_recommendation = product_recommendation.rename(columns={"transaction_id":"number_of_transactions"})

In [39]:
product_recommendation.head()

,product,product_category,number_of_transactions
0,Almond Croissant,Bakery,347
1,Cappuccino,Coffee,1290
2,Carmel syrup,Flavours,561
3,Chocolate Chip Biscotti,Bakery,352
4,Chocolate Croissant,Bakery,636


In [41]:
product_recommendation.to_csv('api/recommendation_objects/popularity_recommendation.csv', index=False)

# Apriori Recommendation Engine

In [42]:
train_basket = (dataset.groupby(["transaction", "product"])["product"].count().reset_index(name="count"))
train_basket.head()

,transaction,product,count
0,1000_0,Dark chocolate,1
1,1000_0,Oatmeal Scone,1
2,1001_8306,Cappuccino,1
3,1001_8306,Carmel syrup,1
4,1002_0,Carmel syrup,1


In [43]:
my_basket = train_basket.pivot_table(index="transaction", columns="product", values="count").fillna(0)
my_basket.head()

product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,,,
1000_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1001_8306,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002_0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1004_5383,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1005_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0


In [44]:
def encode_units(x):
    if x<=0:
        return 0
    if x>0:
        return 1

my_basket_sets = my_basket.applymap(encode_units)
my_basket_sets.head()

product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,,,
1000_0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1001_8306,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1002_0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0
1004_5383,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1005_0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [45]:
frequent_items = apriori(my_basket_sets, min_support=0.05, use_colnames=True)
frequent_items.head(20)

c:\Users\vaish\Desktop\chatbot_LLM_app\.venv\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.115646,(Almond Croissant)
1,0.388889,(Cappuccino)
2,0.191232,(Carmel syrup)
3,0.112623,(Chocolate Chip Biscotti)
4,0.135676,(Chocolate Croissant)
5,0.188964,(Chocolate syrup)
6,0.116024,(Cranberry Scone)
7,0.114135,(Croissant)
8,0.277022,(Dark chocolate)
9,0.209373,(Espresso shot)


In [48]:
rules_basket = association_rules(frequent_items, metric="lift", min_threshold=1)
rules_basket.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Almond Croissant),(Cappuccino),0.115646,0.388889,0.053288,0.460784,1.184874,0.008314,1.133333,0.176432
1,(Cappuccino),(Almond Croissant),0.388889,0.115646,0.053288,0.137026,1.184874,0.008314,1.024775,0.255319
2,(Almond Croissant),(Dark chocolate),0.115646,0.277022,0.057445,0.496732,1.793115,0.025409,1.436567,0.500152
3,(Dark chocolate),(Almond Croissant),0.277022,0.115646,0.057445,0.207367,1.793115,0.025409,1.115717,0.611791
4,(Almond Croissant),(Latte),0.115646,0.382086,0.054422,0.470588,1.231629,0.010235,1.167171,0.212660


In [49]:
rules_basket[rules_basket["antecedents"]=={"Latte"}].sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
71,(Latte),(Sugar Free Vanilla syrup),0.382086,0.200302,0.108844,0.284866,1.422182,0.032311,1.118249,0.480415
33,(Latte),(Carmel syrup),0.382086,0.191232,0.107710,0.281899,1.474121,0.034643,1.126260,0.520509
45,(Latte),(Chocolate syrup),0.382086,0.188964,0.103175,0.270030,1.428997,0.030974,1.111053,0.485842
69,(Latte),(Hazelnut syrup),0.382086,0.171580,0.101285,0.265084,1.544961,0.035727,1.127231,0.570848
53,(Latte),(Croissant),0.382086,0.114135,0.057067,0.149357,1.308605,0.013458,1.041407,0.381651
39,(Latte),(Chocolate Croissant),0.382086,0.135676,0.055178,0.144411,1.064381,0.003338,1.010209,0.097889
67,(Latte),(Ginger Scone),0.382086,0.133409,0.055178,0.144411,1.082472,0.004204,1.012860,0.123300
5,(Latte),(Almond Croissant),0.382086,0.115646,0.054422,0.142433,1.231629,0.010235,1.031236,0.304358
65,(Latte),(Ginger Biscotti),0.382086,0.106198,0.054044,0.141444,1.331890,0.013467,1.041053,0.403272
49,(Latte),(Cranberry Scone),0.382086,0.116024,0.051398,0.134520,1.159416,0.007067,1.021371,0.222518


## Saving in JSON

In [50]:
product_categories = dataset[["product", "product_category"]].drop_duplicates().set_index("product").to_dict()["product_category"]

In [51]:
product_categories

{'Cappuccino': 'Coffee',
 'Jumbo Savory Scone': 'Bakery',
 'Latte': 'Coffee',
 'Chocolate Chip Biscotti': 'Bakery',
 'Espresso shot': 'Coffee',
 'Hazelnut Biscotti': 'Bakery',
 'Chocolate Croissant': 'Bakery',
 'Dark chocolate': 'Packaged Chocolate',
 'Cranberry Scone': 'Bakery',
 'Croissant': 'Bakery',
 'Almond Croissant': 'Bakery',
 'Ginger Biscotti': 'Bakery',
 'Oatmeal Scone': 'Bakery',
 'Ginger Scone': 'Bakery',
 'Chocolate syrup': 'Flavours',
 'Hazelnut syrup': 'Flavours',
 'Carmel syrup': 'Flavours',
 'Sugar Free Vanilla syrup': 'Flavours'}

In [60]:
recommendations_json = {}

antecedents = rules_basket['antecedents'].unique()
for antecedent in antecedents:
    df_rec = rules_basket[rules_basket['antecedents']==antecedent]
    df_rec = df_rec.sort_values('confidence',ascending=False)
    key = "_".join(antecedent)
    recommendations_json[key] = []
    for _, row in df_rec.iterrows():
        rec_objects =row['consequents']
        for rec_object in rec_objects:
            already_exists = False
            for current_rec_object in recommendations_json[key]:
                if rec_object == current_rec_object['product']:
                    already_exists=True
            if already_exists:
                continue
            
            rec = {'product':rec_object, 
                   'product_category':product_categories[rec_object],
                   'confidence': row['confidence']
                  }
            recommendations_json[key].append(rec)

In [61]:
import pprint

In [62]:
pprint.pp(recommendations_json)

{'Almond Croissant': [{'product': 'Dark chocolate',
                       'product_category': 'Packaged Chocolate',
                       'confidence': 0.4967320261437908},
                      {'product': 'Latte',
                       'product_category': 'Coffee',
                       'confidence': 0.47058823529411764},
                      {'product': 'Cappuccino',
                       'product_category': 'Coffee',
                       'confidence': 0.46078431372549017}],
 'Cappuccino': [{'product': 'Sugar Free Vanilla syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.29057337220602525},
                {'product': 'Chocolate syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.2818270165208941},
                {'product': 'Carmel syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.26433430515063167},
                {'product': 'Hazelnut syrup',
           

In [65]:
import json
with open("api/recommendation_objects/apriori_recommendations.json", "w") as json_file:
    json.dump(recommendations_json, json_file)